In [1]:
import numpy as np
from model_2point import *


In [2]:
def getw_rho(wplus,wminus,phiA,phiB,rho):
    wplus = wplus
    wminus = wminus
    wA = (wplus+wminus)/2
    wB = (wplus-wminus)/2
    rhoA = rho(wA,phiA)
    rhoB = rho(wB,phiB)
    return rhoA,rhoB


class model():
    def __init__(self,chi,rho0,ensemble = 'canonical'):
        self.chi = chi
        self.rho0 = rho0
        
        if ensemble == 'canonical':
            self.rho = rhoc
        elif ensemble == 'grand':
            self.rho = rhog
        self.zeta = 1.0
        self.ensemble = ensemble
    def init_canonical(self,phiA):
        self.phiA = phiA
        self.phiB = 1-phiA
    def init_grandcanonical(self,zA,zB):
        # these should be zA = \rho0^{-1} exp(muA) and zB = \rho0^{-1} exp(muB)
        self.phiA = zA
        self.phiB = zB
    def init_weakcompressible(self,zeta):
        self.zeta = zeta
    def dHdw_compressible(self,w):
        wplus = w[:,0]
        wminus = w[:,1]
        rhoA,rhoB = getw_rho(wplus,wminus,self.phiA,self.phiB,self.rho)
        dHdwplus = (rhoA+rhoB)-2/self.chi*wplus
        dHdwminus = (rhoB-rhoA)+2/self.chi*wminus
        return self.rho0*np.vstack((dHdwplus,-dHdwminus)).T
    def dHdw_weakcompressible(self,w):
        wplus = w[:,0]
        wminus = w[:,1]
        wplus = w[:,0]
        wminus = w[:,1]
        rhoA,rhoB = getw_rho(wplus,wminus,self.phiA,self.phiB,self.rho)
        dHdwplus = (rhoA+rhoB)-2/(self.chi+2*self.zeta)*(wplus-self.zeta)
        dHdwminus = (rhoB-rhoA)+2/self.chi*wminus
        return self.rho0*np.vstack((dHdwplus,-dHdwminus)).T
    def dHdw_incompressible(self,w):
        wplus = w[:,0]
        wminus = w[:,1]
        wplus = w[:,0]
        wminus = w[:,1]
        rhoA,rhoB = getw_rho(wplus,wminus,self.phiA,self.phiB,self.rho)
        dHdwplus = (rhoA+rhoB)-1
        dHdwminus = (rhoB-rhoA)+2/self.chi*wminus
        return self.rho0*np.vstack((dHdwplus,-dHdwminus)).T
    def wick(self,w):
        return np.vstack((np.zeros(len(w),dtype = complex)+1j,np.ones(len(w),dtype = complex))).T
    def g(self,w):
        if len(np.shape(w)) == 1:
            w = w.reshape((1,2))
        wplus = w[:,0]
        wminus = w[:,1]
        rhoA,rhoB = getw_rho(wplus,wminus,self.phiA,self.phiB,self.rho)
        g = self.rho0*(rhoA+rhoB-1)
        return g
    def dgdw(self,w):
        wplus = w[:,0]
        wminus = w[:,1]
        rhoA,rhoB = getw_rho(wplus,wminus,self.phiA,self.phiB,self.rho)
        if self.ensemble == 'grand':
            dgdwplus = -(rhoA+rhoB)
            dgdwminus = -(rhoB-rhoA)
            return np.vstack((dgdwplus,dgdwminus)).T
    def getDensity(self,w):
        return self.rho0*calc_density(w,self.rho,self.phiA,self.phiB)
    def getCorrelation(self,w):
        return self.rho0*calcDensity2(w,self.rho,self.phiA,self.phiB)
    def getCorrelation_list(self,wlist):
        return np.array([self.rho0*calcDensity2(w,self.rho,self.phiA,self.phiB) for w in wlist],dtype = complex)    

    def dgldl(self,w,l):
        wplus = w[:,0]
        wminus = w[:,1]
        rhoA,rhoB = getw_rho(wplus,wminus,self.phiA,self.phiB,self.rho)
        rho_plus = rhoA+rhoB
        rho_minus = rhoB-rhoA
        wnew = w.copy()
        wnew[:,0] += l*self.dgdw(w)[:,0]
        wnew[:,1] += l*self.dgdw(w)[:,1]

        if self.ensemble == 'grand':
            wplus_c = wnew[:,0]
            wminus_c = wnew[:,1]
            rhoA_adjust,rhoB_adjust = getw_rho(wplus_c,wminus_c,self.phiA,self.phiB,self.rho)
            dgdl = rhoA_adjust*(rho_plus-rho_minus)+rhoB_adjust*(rho_plus+rho_minus)
            dgdl *= self.rho0*0.25
            return dgdl
    def gl(self,w,l):
        wnew = w.copy()
        wnew[:,0] += l*self.dgdw(w)[:,0]
        wnew[:,1] += l*self.dgdw(w)[:,1]


        return self.g(wnew)




In [23]:


# Example usage:
# Define the function whose root is

phiA= 0.5
phiB = 1-phiA
chi = 1.0
rho0 = 1.0
nx = 10
zA = 1e3
zB = 1e3
tmax = 10
dt = 1e-5
zetalist = [0.0,0.1,0.5,1,2,5,10,100]

# Generate arrays of complex numbers
wplus = np.random.normal(0, 1, nx) + 0j * np.random.normal(0, 1, nx)
wplus = wplus-np.mean(wplus)
wminus = np.random.normal(0, 1, nx) + 0j * np.random.normal(0, 1, nx)
w0 = np.vstack((wplus,wminus)).T
lambda_plus = 1.0
lambda_minus = 1.0
_model = model(chi,rho0,ensemble='grand')
_model.init_grandcanonical(zA,zB)
#model.dHw(w)

model_list = [_model.dHdw_compressible,_model.dHdw_weakcompressible,_model.dHdw_incompressible]
lambdat = np.array([lambda_plus,lambda_minus])
wlist = []
corrlist = []
sde = sde_int(w0,_model.dHdw_compressible,_model.wick)
# sde.initialize_project(_model.g,_model.dgdw)
w,t, = sde.Euler_Maruyama(2*dt,dt,lambdat,SCFT = False)

error = 1
tol= 1e-4
lag_mult = np.ones(nx,dtype = complex)*0.01
w0 = w[-1]

def F(w,wfix,lag_mult,_model):
    return w-wfix-lag_mult*_model.dgdw(wfix)

itter=0
error = 1
tol = 1e-5
lag_mult_old = lag_mult.copy()
while error > tol:
    lag_mult+= -(_model.gl(w0,lag_mult))/_model.dgldl(w0,lag_mult)
    
    error = np.sqrt(np.sum(np.square(lag_mult.real-lag_mult_old.real)+np.square(lag_mult.imag-lag_mult_old.imag)))
    lag_mult_old = lag_mult.copy()
    itter+=1
    if itter == 1000:
        break
print(itter)
print(error)
print(_model.gl(w0,lag_mult))
corrlist.append(_model.getCorrelation_list(w))

# sde = sde_int(w0,_model.dHdw_incompressible,_model.wick)
# w,t, = sde.Euler_Maruyama(tmax,dt,lambdat,SCFT = Fals

48
8.837107320979886e-06
[ 2.22044605e-16-2.13875819e-19j -1.11022302e-16+4.94839295e-19j
  2.22044605e-16-2.44114895e-18j  6.66133815e-16+3.89635156e-20j
 -1.11022302e-16-1.73811161e-18j  5.56669763e-04+7.51563199e-06j
  7.18685047e-03+2.20619724e-04j  1.81854531e-13-1.70891930e-15j
  3.56233519e-04+3.53800226e-06j  1.63659941e-08-1.77642655e-11j]
